In [ ]:
import os, sys
sys.path.append('..')
import numpy as np
# from imageio import imread
from skimage.io import imread
from skimage.transform import resize

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
root_dir = os.path.join('/', 'mnt', 'sdb2', 'Datasets', 'asirra')
trainval_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')

### Load dataset

In [ ]:
from datasets import asirra as dataset
X_trainval, y_trainval = dataset.read_asirra_subset(trainval_dir, one_hot=True)

In [ ]:
val_size = int(trainval_size * 0.2)    # FIXME

train_set = dataset.DataSet(X_trainval[:val_size], y_trainval[:val_size])
val_set = dataset.DataSet(X_trainval[val_size:], y_trainval[val_size:])

In [ ]:
print(train_set.images.shape)
print(train_set.images.min(), train_set.images.max())
print(val_set.images.shape)
print(val_set.images.min(), val_set.images.max())

In [ ]:
print((train_set.labels[:, 1] == 0).sum(), (train_set.labels[:, 1] == 1).sum())
print((val_set.labels[:, 1] == 0).sum(), (val_set.labels[:, 1] == 1).sum())

### Preliminary experiments

In [ ]:
hp_d = dict()
# Mean image
hp_d['image_mean'] = train_set.images.mean(axis=(0, 1, 2))
print(hp_d['image_mean'])
print((train_set.images - hp_d['image_mean']).mean(axis=(0, 1, 2)))

In [ ]:
# FIXME: Regularization hyperparameters
hp_d['weight_decay'] = 0.0005
hp_d['dropout_prob'] = 0.5

# FIXME: Training hyperparameters
hp_d['batch_size'] = 256
hp_d['num_epochs'] = 200

hp_d['augment_train'] = True 
hp_d['augment_pred'] = True 

hp_d['init_learning_rate'] = 0.01
hp_d['momentum'] = 0.9
hp_d['learning_rate_patience'] = 30
hp_d['learning_rate_decay'] = 0.1
hp_d['eps'] = 1e-8

# FIXME: Evaluation hyperparameters
hp_d['score_threshold'] = 1e-4

In [ ]:
import tensorflow as tf
from models.nn import AlexNet as ConvNet
from learning.optimizers import MomentumOptimizer as Optimizer
from learning.evaluators import AccuracyEvaluator as Evaluator

In [ ]:
tf.reset_default_graph()
graph = tf.get_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(graph=graph, config=config)

model = ConvNet([227, 227, 3], 2, **hp_d)
evaluator = Evaluator()
optimizer = Optimizer(model, train_set, evaluator, val_set=val_set, **hp_d)

In [ ]:
print(-np.log(0.5))
train_results = optimizer.train(sess, details=True, verbose=True, **hp_d)    # FIXME